# Setting up my own pipeline 786

In [7]:
import numpy as np
import torch

## Read Data

In [3]:
import pandas as pd

# Read just the first row as a DataFrame
df_first = pd.read_csv("/home/feolalab/Desktop/EUCLID-hyperelasticity-NN/my_data/Cube/jobs/outputs/F_last")

# If your file has a header row you want to treat as data, keep header=None.
# If it already has column names in row 1 and you want those as column names:
# df_first = pd.read_csv("file.txt", nrows=1)

print(df_first.head())


   1  1.00198  0.000370964  -0.0102491  0.000370964.1  1.00198.1  \
0  2  1.00138     0.000164   -0.006888      -0.000303    1.00194   
1  3  1.00104     0.000081   -0.004466      -0.000219    1.00207   
2  4  1.00089     0.000045   -0.002534      -0.000129    1.00212   
3  5  1.00082     0.000015   -0.000823      -0.000042    1.00214   
4  6  1.00082    -0.000015    0.000823       0.000042    1.00214   

   -0.0102491.1  0.000866093  0.000866093.1  0.994611  
0     -0.010181     0.000096       0.000846  0.995574  
1     -0.010703     0.000090       0.000831  0.995760  
2     -0.011051     0.000034       0.000829  0.995884  
3     -0.011222     0.000012       0.000830  0.995929  
4     -0.011222    -0.000012       0.000830  0.995929  


In [8]:
#Read position
displacement_last = pd.read_csv("/home/feolalab/Desktop/EUCLID-hyperelasticity-NN/my_data/Cube/jobs/outputs/displacement_last")

# Drop the first column, then convert to an (N×9) Python list:
displacement =torch.tensor( displacement_last.iloc[:, 1:].to_numpy())   # shape = (N, 9)

displacement.shape

torch.Size([1330, 3])

In [ ]:

#Read position
F_last = pd.read_csv("/home/feolalab/Desktop/EUCLID-hyperelasticity-NN/my_data/Cube/jobs/outputs/F_last")

# Drop the first column, then convert to an (N×9) Python list:
F_last = torch.tensor(F_last.iloc[:, 1:].to_numpy())   # shape = (N, 9))

# Now `list_9` is a list of N rows, each row itself a list of length 9.
F_last.shape

torch.Size([999, 9])

In [15]:

#Read connectivity
connectivity = pd.read_csv("/home/feolalab/Desktop/EUCLID-hyperelasticity-NN/my_data/Cube/jobs/outputs/connectivity")

# Drop the first column, then convert to an (N×9) Python list:
connectivity = torch.tensor(connectivity.iloc[:, 1:].to_numpy())   # shape = (N, 9))

# Now `list_9` is a list of N rows, each row itself a list of length 9.
connectivity.shape

torch.Size([999, 8])

In [20]:
#Read volume
vol = pd.read_csv("/home/feolalab/Desktop/EUCLID-hyperelasticity-NN/my_data/Cube/jobs/outputs/vol_last")

# Drop the first column, then convert to an (N×9) Python list:
vol = torch.tensor(vol.iloc[:, 1:].to_numpy())   # shape = (N, 9))

vol.shape

torch.Size([999, 1])

In [16]:
numNodes=displacement.shape[0]
numElements=F_last.shape[0]
num_nodes_per_element=8
numNodes, numElements

(1330, 999)

## Set up architecture of network

In [1]:
from model import *

model = ICNN3D(n_input=9,
                n_hidden=[64,64,64],
                n_output=1,
                use_dropout=True,
                dropout_rate=0.2)

In [6]:
from torchinfo import summary
model = model.to("cuda")
summary(model)


Layer (type:depth-idx)                   Param #
ICNN3D                                   --
├─ModuleDict: 1-1                        --
│    └─Linear: 2-1                       640
│    └─convexLinear: 2-2                 4,096
│    └─convexLinear: 2-3                 4,096
│    └─convexLinear: 2-4                 64
├─ModuleDict: 1-2                        --
│    └─Linear: 2-5                       640
│    └─Linear: 2-6                       640
│    └─convexLinear: 2-7                 9
Total params: 10,185
Trainable params: 10,185
Non-trainable params: 0

## Virtual Fields

In [ ]:
v_x_star1 = torch.zeros_like(displacement[:,0]) 
v_y_star1 = -displacement[:,1] #corrected
v_z_star1 = torch.zeros_like(displacement[:,0]) 


virtual_displacement1 = torch.stack([v_x_star1, v_y_star1, v_z_star1], dim=1)  #torch.Size([Nodes, 2])

zeros = torch.zeros_like(displacement[:, 0])
twos = torch.ones_like(displacement[:, 0]) * 2

# ∂v_x/∂x, ∂v_x/∂y, ∂v_y/∂x, ∂v_y/∂y
# Row-major order: [vx,x, vx,y, vx,z, vy,x, vy,y, vy,z, vz,x, vz,y, vz,z]
gradient_virtual_displacement1 = torch.stack([
    zeros,  # ∂v_x/∂x
    zeros,  # ∂v_x/∂y
    zeros,  # ∂v_x/∂z
    zeros,  # ∂v_y/∂x
    twos,   # ∂v_y/∂y
    zeros,  # ∂v_y/∂z
    zeros,  # ∂v_z/∂x
    zeros,  # ∂v_z/∂y
    zeros   # ∂v_z/∂z
], dim=1)  # shape: [N, 9]


## Internal Virtual Work

$$
\text{IVW}_k(\mathcal{Q}, \mathbf{v}^*) 
= V_k \, \xi^{*k}_{ij} \, \sigma^k_{ij}(\mathcal{Q}) 
= V_k \, \xi^{*k}_{ij} \left( \frac{1}{J_k} P^k_{im} F^k_{jm} \right)
$$
Where:
* k is each element

In [22]:
F_mat = torch.stack([
    torch.stack([F_last[:,0:1], F_last[:,1:2],F_last[:,2:3]], dim=-1),
    torch.stack([F_last[:,3:4], F_last[:,4:5],F_last[:,5:6]], dim=-1),
    torch.stack([F_last[:,6:7], F_last[:,7:8],F_last[:,8:9]], dim=-1)
], dim=-2).squeeze() #torch.Size([2752, 2, 2]) after squeeze 
P_mat=torch.zeros_like(F_mat)
J = torch.det(F_mat)         # [nElem]
F_mat.shape

torch.Size([999, 3, 3])

In [36]:
connectivity.shape, a, connectivity[:,a].shape

(torch.Size([999, 8]), 0, torch.Size([999]))

In [39]:
connectivity, connectivity.shape


(tensor([[   9,   10,  514,  ...,  118,  604,  603],
         [  10,   11,  515,  ...,  119,  605,  604],
         [  11,   12,  516,  ...,  120,  606,  605],
         ...,
         [1329, 1330,  352,  ...,  601,   97,   96],
         [1330, 1331,  351,  ...,  602,   98,   97],
         [1331,  278,   62,  ...,   63,    7,   98]]),
 torch.Size([999, 8]))

In [31]:
gradient_virtual_displacement1.shape

torch.Size([1330, 9])

In [ ]:
 #connectivity is a list of 3 1D-matrix. Each matrix contains integers.
#Length of each matrix is equal to numElements. 
#For 2D-triangular elements with a={0,1,2}:
#connectivity[a][e]: is the index of the 'a'-th node of the 'e'-th element.
#Note that node indices range from 0 to (numNodes-1).

In [ ]:
ewk=torch.zeros(numNodes)
iwk=torch.zeros(numNodes)


#Compute internal work
for a in range(num_nodes_per_element): #this is 3
    print(a)

    # # Mapping from **nodes to elements**
    element_ivf1 = gradient_virtual_displacement1[connectivity[a]]  #VD is in each node. 

    # --- form Cauchy stress: σ = (1/J) P · Fᵀ ---
    
    sigma = (P_mat @ F_mat.transpose(-2,-1)) / J.view(-1,1,1) # [nElem,2,2]
                                
    # --- flatten σᵢⱼ in the standard row‐major order [11,12,21,22] ---
    sigma_flat = sigma.view(-1,9)   # [nElem,9]	
    #print(sigma_flat.shape)				
    correct_internal_virtual_work= (sigma_flat* (element_ivf1)).sum(dim=1)* vol

    iwk.index_add_(0,connectivity[a],correct_internal_virtual_work) #Nodes 

0


IndexError: index 117 is out of bounds for dimension 0 with size 9

In [34]:
connectivity[a]

tensor([  9,  10, 514, 513, 117, 118, 604, 603])

In [28]:
sigma_flat.shape, element_ivf1.shape, vol.shape

(torch.Size([999, 9]), torch.Size([8, 9]), torch.Size([999, 1]))

## Losses

## Simple training loop

## Test